In [23]:
%%time
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
plt.style.use('seaborn')
pd.set_option("display.max_rows", 10000)
pd.set_option("display.max_columns", 10000)
df_train = pd.read_csv('event--training.csv')
df_test = pd.read_csv('event--test.csv')
df_train.drop(['case description', 'event org:resource'], axis=1)
#df_train['event time:timestamp'] = pd.to_datetime(df_train['event time:timestamp'])
df_train.set_index(["case concept:name", "eventID "], inplace=True)
#df_train['seconds'] = [time.mktime(t.timetuple()) for t in df_train['event time:timestamp']]
df_train["Completion Time"] = 0
df_train.replace(0,np.NaN, inplace=True)



def EventTime(data):
    memory = {}
    for i, j in data[["event concept:name"]].iterrows():
        if j[0] not in memory.keys() and data.loc[i]["event lifecycle:transition"] == "start":
            memory[j[0]] = i
        elif j[0] in memory.keys():
            #Calculate time
            #print(memory[j[0]],",", i)
            time = pd.to_datetime(data.loc[i]["event time:timestamp"], dayfirst=True) - pd.to_datetime(data.loc[memory[j[0]]]["event time:timestamp"], dayfirst=True)           # print(time)
            data.loc[i,"Completion Time"] = time

            del memory[j[0]]
    return data


for idx, new_df in df_train.groupby(level=0):
    #print(idx)
    #EventTime(new_df)
    df_train.loc[idx] = EventTime(new_df)
    
df_train.head(100)

Wall time: 9.91 s


case description case Class  \
case concept:name eventID                                               
1                 0             Simulated process instance      Print   
                  1             Simulated process instance      Print   
                  2             Simulated process instance      Print   
                  3             Simulated process instance      Print   
                  4             Simulated process instance      Print   
                  5             Simulated process instance      Print   
                  6             Simulated process instance      Print   
                  7             Simulated process instance      Print   
                  8             Simulated process instance      Print   
                  9             Simulated process instance      Print   
                  10            Simulated process instance      Print   
                  11            Simulated process instance      Print   
                  12            Simulated process instance      Print   
                  13            Simulated process instance      Print   
                  14            Simulated process instance      Print   
                  15            Simulated process instance      Print   
                  16            Simulated process instance      Print   
                  17            Simulated process instance      Print   
                  18            Simulated process instance      Print   
                  19            Simulated process instance      Print   
                  20            Simulated process instance      Print   
                  21            Simulated process instance      Print   
                  22            Simulated process instance      Print   
                  23            Simulated process instance      Print   
                  24            Simulated process instance      Print   
                  25            Simulated process instance      Print   
                  26            Simulated process instance      Print   
                  27            Simulated process instance      Print   
                  28            Simulated process instance      Print   
                  29            Simulated process instance      Print   
                  30            Simulated process instance      Print   
                  31            Simulated process instance      Print   
                  32            Simulated process instance      Print   
                  33            Simulated process instance      Print   
                  34            Simulated process instance      Print   
2                 51539607552   Simulated process instance  Copy/Scan   
1                 35            Simulated process instance      Print   
                  37            Simulated process instance      Print   
                  36            Simulated process instance      Print   
                  39            Simulated process instance      Print   
                  38            Simulated process instance      Print   
3                 98784247808   Simulated process instance      Print   
1                 41            Simulated process instance      Print   
                  40            Simulated process instance      Print   
                  42            Simulated process instance      Print   
                  43            Simulated process instance      Print   
                  44            Simulated process instance      Print   
                  45            Simulated process instance      Print   
                  46            Simulated process instance      Print   
                  47            Simulated process instance      Print   
                  49            Simulated process instance      Print   
                  48            Simulated process instance      Print   
                  50            Simulated process instance      Print   
                  51       

In [24]:
Average_time = df_train.groupby(['event concept:name'])[['Completion Time']].mean()
Average_time

,Completion Time
event concept:name,
A/D Conversion,NaT
AM Screening,NaT
Accumulate Images,NaT
Apply Heat,NaT
Apply Negative Charge on Drum,NaT
Apply Pressure,NaT
Calc Quantization Error,NaT
Calc Total Neighbor Quant Error,NaT
Coat Light Toner on Drum,NaT
